# Forecasting Example

In this notebook, we have developed code the show an example of how to obtain and plot both mean and interval forecast of a fitted UnnobservedComponentGAS model.

## Import packages

In [5]:
using Plots, CSV, DataFrames

import Pkg
Pkg.activate(".")
Pkg.instantiate()

include("src/UnobservedComponentsGAS.jl")

  Activating project at `c:\Users\matheuscn.ELE.000\Documents\LAMPS\Pacotes\GAS_LAMPSPUC\UnobservedComponentsGAS`
┌ Warning: The project dependencies or compat requirements have changed since the manifest was last resolved.
│ It is recommended to `Pkg.resolve()` or consider `Pkg.update()` if necessary.
└ @ Pkg.API C:\workdir\usr\share\julia\stdlib\v1.8\Pkg\src\API.jl:1535


Main.UnobservedComponentsGAS

## Reading Energy Consumption data

In [ ]:
data = CSV.read("examples/data/energy_consumption.csv", DataFrame)

steps_ahead = 24
y_train     = data[1:end-steps_ahead, 2]
y_val       = data[end-steps_ahead+1:end, 2] 
dates_train = data[1:end-steps_ahead, 1]
dates_val   = data[end-steps_ahead+1:end, 1]

plot(title = "Energy Consumption")
plot!(dates_train, y_train, label = "train")
plot!(dates_val, y_val, label = "validation")

## Define a Score-Driven Model

Model specifications:
 - Distribution: LogNormal
 - Only the first parameter as time varying
 - d = 1
 - 500 scenarios to obtain the forecast
 - level component: random walk with slope process
 - seasonality component: deterministic with 12 seasonal periods (monthly seasonality)
 - ar component: missing

Since we have only the first parameter as time varying, we can pass a string "dynamic" instead of a vector of strings ["dynamic", ""].
 - *Example*: we are able to define level = "random walk slope" instead of level = ["random walk slope", ""]


In [ ]:
dist                    = UnobservedComponentsGAS.LogNormalDistribution()
time_varying_parameters = [true, false]
d                       = 1.0
num_scenarious          = 500

level       = "random walk slope"
seasonality = "deterministic 12"
ar          = missing

model = UnobservedComponentsGAS.GASModel(dist, time_varying_parameters, d, level,seasonality, ar)

## Fit and Forecast the specified model

In [ ]:
fitted_model = UnobservedComponentsGAS.fit(model, y_train; robust = false)
forec        = UnobservedComponentsGAS.predict(model, fitted_model, y_train, steps_ahead, num_scenarious)

## Plot Mean Forecast with Confidence Intervals

In [ ]:
plot(title = "Mean Forecast with Confidence Intervals")
plot!(dates_val, forec["intervals"]["95"]["lower"], fillrange = forec["intervals"]["95"]["upper"], fillalpha = 0.15, color = :grey, label = "95% Confidence band")
plot!(dates_val, forec["intervals"]["80"]["lower"], fillrange = forec["intervals"]["80"]["upper"], fillalpha = 0.15, color = :darkgrey, label = "80% Confidence band")
plot!(dates_val, forec["intervals"]["95"]["upper"], label = "", color = :grey)
plot!(dates_val, forec["intervals"]["80"]["upper"], label = "", color = :darkgrey)
plot!(dates_val, y_val, label = "time series", color = :blue)
plot!(dates_val, forec["mean"], label = "mean forecast", color = :red)

## Plot Mean Forecast with Scenarios

In [ ]:
plot(title = "Mean Forecast with Scenarios")
plot!(dates_val, forec["scenarios"], label = "", color = :grey60)
plot!(dates_val, y_val, label = "time series", color = :blue)
plot!(dates_val, forec["mean"], label = "mean forecast", color = :red)